# Sending Lotus Notes Email - example taken from SO [example](http://stackoverflow.com/questions/26065191/send-mail-in-lotus-notes-using-python)

### Dependencies:

- Python 3.x
- pywin32
- need to know your mail server location and mail database file location
    - these typically can be found in your notes.ini file located in your Notes local "Data" folder

In [2]:
import os, uuid
import itertools as it
from getpass import getpass

from win32com.client import DispatchEx
import pywintypes # for exception

# Get password from user without echoing/displaying it
pw = getpass('Enter Notes password: ')

def send_mail(subject,body_text,sendto,copyto=None,blindcopyto=None,
              attach=None):
    session = DispatchEx('Lotus.NotesSession')
    session.Initialize(pw)

    
    # For WRP:
    # server_name = 'NSHAMM04/HAM/S_SRV/AM/HONDA'
    # db_name = 'mail/hamaqwrp.nsf'
    server_name = 'NSHAMM01/HAM/S_SRV/AM/HONDA'  # At DOS prompt: ping NSHAMM01 to get ip address
    db_name = 'mail/ma17151.nsf'

    db = session.getDatabase(server_name, db_name)
    if not db.IsOpen:
        try:
            db.Open()
        except pywintypes.com_error:
            print( 'could not open database: {}'.format(db_name) )

    doc = db.CreateDocument()
    doc.ReplaceItemValue("Form","Memo")
    doc.ReplaceItemValue("Subject",subject)

    # assign random uid because sometimes Lotus Notes tries to reuse the same one
    uid = str(uuid.uuid4().hex)
    doc.ReplaceItemValue('UNIVERSALID',uid)

    # "SendTo" MUST be populated otherwise you get this error: 
    # 'No recipient list for Send operation'
    doc.ReplaceItemValue("SendTo", sendto)

    if copyto is not None:
        doc.ReplaceItemValue("CopyTo", copyto)
    if blindcopyto is not None:
        doc.ReplaceItemValue("BlindCopyTo", blindcopyto)

    # body
    body = doc.CreateRichTextItem("Body")
    body.AppendText(body_text)

    # attachment 
    if attach is not None:
        attachment = doc.CreateRichTextItem("Attachment")
        for att in attach:
            attachment.EmbedObject(1454, "", att, "Attachment")

    # save in `Sent` view; default is False
    doc.SaveMessageOnSend = True
    doc.Send(False)

if __name__ == '__main__':
    subject = "test subject"
    body = "test body - sent by a Python bot"
    sendto = ['daniel_j_kim@hna.honda.com']
    files = ['D:\\temp\\2016-10-18base_name.txt']
    attachment = it.takewhile(lambda x: os.path.exists(x), files)

    send_mail(subject, body, sendto, attach=attachment)

Enter Notes password: ········
